In [1]:
print("OK!")

OK!


In [2]:
from langchain import PromptTemplate

In [3]:
from langchain.chains import RetrievalQA

In [4]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

c:\Users\USER\chatbot\my_env\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [5]:
PINECONE_API_KEY = "pcsk_29fdB4_7hYK4c2LD1KDztd1cyuyyaSFnDTx2f3meEFy755scd77xnqF7Tt1aDg5vBttCXr"
PINECONE_API_ENV = "quickstart"

In [6]:
#Extarch data from the pdf
def load_pdf(data):
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [7]:
extracted_data = load_pdf("data/")

In [8]:
# extracted_data

In [9]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)
    
    return text_chunks

In [10]:
text_chunks = text_split(extracted_data)
print("length of my chunk: ", len(text_chunks))

length of my chunk:  5860


In [11]:
#download embedding model 
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [12]:
embeddings = download_hugging_face_embeddings()

C:\Users\USER\AppData\Local\Temp\ipykernel_15584\2040965682.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [13]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [14]:
query_result = embeddings.embed_query("Hello Kenya")
print("Length", len(query_result))

Length 384


In [16]:
from langchain.vectorstores import Pinecone
from pinecone import Pinecone as PineconeClient
import os

# Your API key and environment
PINECONE_API_KEY = "pcsk_29fdB4_7hYK4c2LD1KDztd1cyuyyaSFnDTx2f3meEFy755scd77xnqF7Tt1aDg5vBttCXr"
PINECONE_ENV = "us-east-1"  # Make sure this matches your Pinecone environment

# Set environment variables
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["PINECONE_ENV"] = PINECONE_ENV

# Initialize Pinecone client
pc = PineconeClient(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_ENV
)

# Create or connect to an index
index_name = "medical-chatbot"

# Check if index already exists
if index_name not in pc.list_indexes().names():
    # Create index if it doesn't exist
    pc.create_index(
        name=index_name,
        dimension=384,  # Changed to 384 for all-MiniLM-L6-v2 embeddings
        metric="cosine"
    )

# Get the index
index = pc.Index(index_name)

# Create vector store using LangChain's Pinecone integration
docsearch = Pinecone.from_documents(
    documents=text_chunks,
    embedding=embeddings,
    index_name=index_name,
    namespace="medical"
)

In [17]:
# To query an existing index
docsearch = Pinecone.from_existing_index(
    index_name=index_name,
    embedding=embeddings,
    namespace="medical"  # Make sure to use the same namespace you used when creating
)

# Perform similarity search
query = "What are allergies"
docs = docsearch.similarity_search(
    query=query,
    k=3  # Number of results to return
)

# Print results in a more readable format
print("\nSearch Results for:", query)
print("-" * 50)
for i, doc in enumerate(docs, 1):
    print(f"\nResult {i}:")
    print(f"Content: {doc.page_content}")
    print(f"Source: {doc.metadata.get('source', 'Not specified')}")
    print("-" * 30)

# Optional: If you want to get both documents and their similarity scores
docs_and_scores = docsearch.similarity_search_with_score(
    query=query,
    k=3
)

print("Result", docs)

# # Print results with scores
# print("\nResults with Similarity Scores:")
# print("-" * 50)
# for i, (doc, score) in enumerate(docs_and_scores, 1):
#     print(f"\nResult {i} (Similarity Score: {score:.4f}):")
#     print(f"Content: {doc.page_content}")
#     print(f"Source: {doc.metadata.get('source', 'Not specified')}")
#     print("-" * 30)


Search Results for: What are allergies
--------------------------------------------------

Result 1:
Content: Purpose
Allergy is a reaction of the immune system. Nor-
mally, the immune system responds to foreign microor-
ganisms and particles, like pollen or dust, by producing
specific proteins called antibodies that are capable of
binding to identifying molecules, or antigens, on the
foreign organisms. This reaction between antibody and
antigen sets off a series of reactions designed to protect
the body from infection. Sometimes, this same series of
Source: data\Medical_book.pdf
------------------------------

Result 2:
Content: Purpose
Allergy is a reaction of the immune system. Nor-
mally, the immune system responds to foreign microor-
ganisms and particles, like pollen or dust, by producing
specific proteins called antibodies that are capable of
binding to identifying molecules, or antigens, on the
foreign organisms. This reaction between antibody and
antigen sets off a series of 

In [18]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Answer only health related questions from the user, If it is outside this don't respond.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [19]:
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [20]:
llm = CTransformers(model="llama-2-7b-chat.ggmlv3.q4_0.bin",
                    model_type="llama",
                    config={"max_new_tokens": 512,
                            'temperature': 0.8}
                    )

In [21]:
qa=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

In [ ]:
while True:
    user_input = input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])

C:\Users\USER\AppData\Local\Temp\ipykernel_15584\495682926.py:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result=qa({"query": user_input})


Response :  The virus that causes AIDS is HIV (Human Immunodeficiency Virus).
Response :  Allergy is an overreaction of the immune system to a harmless substance, such as pollen, dust, or a specific food. This overreaction can cause a variety of symptoms, including itchy eyes, runny nose, congestion, coughing, wheezing, and difficulty breathing. Allergies can also lead to more serious conditions, such as anaphylaxis, which can be life-threatening.
